In [1]:
# Imports
import imageio
import numpy as np
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import os
from scipy.signal import savgol_filter

In [2]:
# Constants
posCoord   = (50, 60)  # 'position' starting coordinates
posChars   = 9*3 + 2   # number of 'position' chars
posNums    = 3         # number of 'position' entries
angleCoord = (50, 74)  # 'angle' starting coordinates
angleChars = 6*2 + 1   # number of 'angle' chars
angleNums  = 2         # number of 'angle' entries
velCoord   = (50, 88)  # 'velocity' starting coordinates
velChars   = 7         # number of 'velocity' chars
velNums    = 1         # number of 'velocity' entries
dt         = 1 / 29.97 # timestep

In [3]:
# Trajectory videos
trajectoryVideos = [
    'Gameplay/Steady_Dive_0deg_Trim.mp4',
    'Gameplay/Steady_Dive_10deg_Trim.mp4',
    'Gameplay/Steady_Dive_20deg_Trim.mp4',
    'Gameplay/Steady_Dive_30deg_Trim.mp4',
    'Gameplay/Steady_Dive_34deg_Trim.mp4',
    'Gameplay/Steady_Dive_35deg_Trim.mp4',
    'Gameplay/Steady_Dive_36deg_Trim.mp4',
    'Gameplay/Steady_Dive_40deg_Trim.mp4',
    'Gameplay/Steady_Dive_50deg_Trim.mp4',
    'Gameplay/Steady_Dive_60deg_Trim.mp4',
    'Gameplay/Steady_Dive_70deg_Trim.mp4',
    'Gameplay/Steady_Dive_80deg_Trim.mp4',
    'Gameplay/Steady_Dive_90deg_Trim.mp4',
    'Gameplay/Dive_Level_1_Trim.mp4',
    'Gameplay/Dive_Level_2_Trim.mp4'
]

In [4]:
# Load in classifier
clf = pickle.load(open('apexOCR.db', 'rb'))

In [5]:
# Extract char image from video frame
def getCharImage(frame, pos, N):
    # Char properties
    charWidth  = 8  # character width
    charHeight = 12 # character height
    
    # Calculate coordinates
    xStart = pos[0] + charWidth*N
    xStop  = xStart + charWidth
    yStart = pos[1]
    yStop  = yStart + charHeight
    
    # Extract image
    return frame[yStart:yStop, xStart:xStop]

In [6]:
# Extract data from image
def getData(frame, pos, N, M, clf):
    # Extract raw chars
    chars = ''
    for i in range(N):
        chars += clf.predict(getCharImage(frame, pos, i).reshape(1,-1))[0]
    
    # Convert to numbers
    return [float(x) for x in chars.split()[0:M]]

In [7]:
# Iterate through each trajectory video
for videoLoc in tqdm(trajectoryVideos, 'Parsing Trajectory Videos', leave=False):
    # Import video
    vid = imageio.get_reader(videoLoc, 'ffmpeg')

    # Iterate through video frames
    tNow = 0
    t    = []
    traj = {
        'xPos':  [],
        'yPos':  [],
        'zPos':  [],
        'pitch': [],
        'yaw':   [],
        'vel':   []
    }
    for frame in vid:
        # Calculate mean frame
        frameMean = np.mean(frame, 2)

        # Extract data
        try:
            # Run OCR
            posData   = getData(frameMean, posCoord, posChars, posNums, clf)
            velData   = getData(frameMean, velCoord, velChars, velNums, clf)
            angleData = getData(frameMean, angleCoord, angleChars, angleNums, clf)        
            
            # Store results
            traj['xPos'].append(posData[0])
            traj['yPos'].append(posData[1])
            traj['zPos'].append(posData[2])
            traj['vel'].append(velData[0])
            traj['pitch'].append(-angleData[0])
            traj['yaw'].append(angleData[1])
            t.append(tNow)
        except:
            pass
        
        # Increment time
        tNow += dt

    # Create DataFrame
    results = pd.DataFrame(traj, t)
    results.index.name = 't'
    
    # Savgol filter settings
    filter_pts = 31
    poly_order = 1
    dt = results.index.to_series().diff().median()
    
    # Calculate flight path angle
    xVel = savgol_filter(results['xPos'], filter_pts, poly_order, 1, dt)
    yVel = savgol_filter(results['yPos'], filter_pts, poly_order, 1, dt)
    zVel = savgol_filter(results['zPos'], filter_pts, poly_order, 1, dt)
    vel  = np.sqrt(xVel**2 + yVel**2 + zVel**2)
    results['gamma'] = np.rad2deg(np.arcsin(zVel / vel))
    
    # Calculate acceleration
    results['acc'] = savgol_filter(results['vel'], filter_pts, poly_order, 1, dt)

    # Save results
    fileName = os.path.splitext(os.path.split(videoLoc)[1])[0].replace('_Trim','')
    if not os.path.isdir('Trajectories'):
        os.mkedir('Trajectories')
    results.to_csv(os.path.join('Trajectories', fileName + '.csv'))